<a href="https://colab.research.google.com/github/MohammadAghaei1/Generative-AI/blob/main/GANs_%26_VAEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, BatchNormalization, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam

# **Loading dataset**

In [ ]:
# Load MNIST dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=3)

# Check the shape of the data
print(X_train.shape)


**Checking activation of GPU**

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cpu


# **Making Generator**

In [ ]:
def build_generator():
    generator = Sequential()

    # Start with a dense layer that outputs a 7x7x128 tensor
    generator.add(Dense(128 * 7 * 7, input_dim=100))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(BatchNormalization(momentum=0.8))
    generator.add(Reshape((7, 7, 128)))  # Reshape to (7, 7, 128)

    # Upsample to 14x14
    generator.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(BatchNormalization(momentum=0.8))

    # Upsample to 28x28
    generator.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(BatchNormalization(momentum=0.8))

    # Final layer to get the output to 28x28x1
    generator.add(Conv2DTranspose(1, kernel_size=3, strides=1, padding='same', activation='tanh'))

    noise = Input(shape=(100,))
    img = generator(noise)

    return generator(noise, img)


In [ ]:
generator.summary()

# **Making Discriminator**

In [ ]:
def build_discriminator():

  discriminator = Sequential()
  discriminator.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(28,28,1), padding="same"))
  discriminator.add(LeakyReLU(alpha=0.2))
  discriminator.add(Dropout(0.25))

  discriminator.add(Conv2D(64, kernel_size=3, strides=2,padding="same"))
  discriminator.add(ZeroPadding2D(padding=((0,1),(0,1))))
  discriminator.add(BatchNormalization(momentum=0.8))

  discriminator.add(LeakyReLU(alpha=0.2))
  discriminator.add(Dropout(0.25))

  discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
  discriminator.add(BatchNormalization(momentum=0.8))
  discriminator.add(LeakyReLU(alpha=0.2))
  discriminator.add(Dropout(0.25))

  discriminator.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
  discriminator.add(BatchNormalization(momentum=0.8))
  discriminator.add(LeakyReLU(alpha=0.2))
  discriminator.add(Dropout(0.25))

  discriminator.add(Flatten())
  discriminator.add(Dense(1, activation='sigmoid'))

  img = Input(shape=(28,28,1))
  probability = discriminator(img)

  return discriminator(inputs=img, outputs=probability)

In [ ]:
discriminator.summary()

# **Making Gan**

In [ ]:
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# build discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# For the combined model we will only train the generator
discriminator.trainable = False

# Build the generator
generator = build_generator()

z = Input(shape=(100,))
img = generator(z)

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(inputs=z, outputs=valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
combined.summary()

# **Traning**

In [ ]:
def train(epochs, batch_size=128, save_interval=100):

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Sample noise and generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)

        # Train the discriminator
        D_loss_real = discriminator.train_on_batch(real_imgs, valid)
        D_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
        D_loss = 0.5 * np.add(D_loss_real, D_loss_fake)

        # Train the generator
        g_loss = combined.train_on_batch(noise, valid)

        # printing progress
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %(epoch, D_loss[0], 100*D_loss[1], g_loss))

        if epoch % save_interval == 0:
            plot_generated_images(epoch, generator)
